<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/survival_outcomes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
# Fragility Metrics Toolkit: Survival Outcomes (Cox Regression / Hazard Ratios)
# 25-NOV-2025
# Fully aligned with FRAGILITY_METRICS.md v10.2 §3.11 SFQ and §4.9 SRQ
#
# Single mode: HR (hazard ratio) + 95% CI
#
# Output: p (approximate), fr (SFQ), nb (SRQ)
#
# Formulas (exact v10.2):
# - SFQ = |z_HR - 1.96| / (1 + |z_HR - 1.96|)
# - SRQ = |ln(HR)| / (1 + |ln(HR)|)
#
# Common applications:
# - Overall survival (OS)
# - Progression-free survival (PFS)
# - Disease-free survival (DFS)
# - Time to heart failure hospitalization
# - Time to cardiovascular death
# - Any Cox regression time-to-event analysis
#
# IF YOU USE THIS CALCULATOR PLEASE CITE:
# Heston, T. F. (2025). Fragility Metrics Toolkit [Software]. Zenodo. https://doi.org/10.5281/zenodo.17254763
#
# © Thomas F. Heston 2025. CC-BY 4.0

import numpy as np
from scipy.stats import norm

ALPHA = 0.05
Z_CRIT = 1.96  # Two-sided critical value at α = 0.05

# ==================== Core calculation ====================
def calculate_survival_metrics(HR, ci_lower, ci_upper):
    """
    Calculate p-fr-nb triplet for survival outcomes from hazard ratio and 95% CI.

    Parameters:
    -----------
    HR : float
        Hazard ratio from Cox regression
    ci_lower : float
        Lower bound of 95% confidence interval for HR
    ci_upper : float
        Upper bound of 95% confidence interval for HR

    Returns:
    --------
    dict with keys:
        - p: approximate two-sided p-value
        - SFQ: Survival Fragility Quotient (fr)
        - SRQ: Survival Robustness Quotient (nb)
        - z_HR: Cox regression z-statistic
        - ln_HR: natural log of HR
        - SE_ln_HR: standard error of ln(HR)
    """

    # Validation
    if HR <= 0:
        raise ValueError("HR must be positive")
    if ci_lower <= 0 or ci_upper <= 0:
        raise ValueError("CI bounds must be positive")
    if ci_lower >= ci_upper:
        raise ValueError("CI lower bound must be less than upper bound")
    if not (ci_lower <= HR <= ci_upper):
        raise ValueError("HR must be within the 95% CI bounds")

    # Step 1: Calculate ln(HR) and SE
    ln_HR = np.log(HR)
    ln_ci_lower = np.log(ci_lower)
    ln_ci_upper = np.log(ci_upper)

    # SE from CI width (CI = estimate ± 1.96 × SE)
    SE_ln_HR = (ln_ci_upper - ln_ci_lower) / (2 * Z_CRIT)

    # Step 2: Calculate z-statistic (Cox regression z-score)
    z_HR = ln_HR / SE_ln_HR

    # Step 3: Calculate SFQ (fragility)
    # Distance from z to critical value (1.96 for two-sided α = 0.05)
    distance_to_boundary = abs(abs(z_HR) - Z_CRIT)
    SFQ = distance_to_boundary / (1 + distance_to_boundary)

    # Step 4: Calculate SRQ (robustness)
    # Distance from neutrality (HR = 1 → ln(HR) = 0)
    SRQ = abs(ln_HR) / (1 + abs(ln_HR))

    # Step 5: Approximate p-value (two-sided)
    p_val = 2 * (1 - norm.cdf(abs(z_HR)))

    return {
        "p": p_val,
        "SFQ": SFQ,
        "SRQ": SRQ,
        "z_HR": z_HR,
        "ln_HR": ln_HR,
        "SE_ln_HR": SE_ln_HR,
        "HR": HR,
        "ci_lower": ci_lower,
        "ci_upper": ci_upper
    }

# ==================== Output formatter ====================
def print_results(results):
    """Pretty-print p-fr-nb results for survival outcomes"""
    print("\n" + "="*70)
    print("SURVIVAL OUTCOMES CALCULATOR – FRAGILITY_METRICS v10.2")
    print("="*70)
    print(f"Hazard Ratio (HR) = {results['HR']:.3f}")
    print(f"95% CI = [{results['ci_lower']:.3f}, {results['ci_upper']:.3f}]")
    print(f"ln(HR) = {results['ln_HR']:.4f}")
    print(f"SE[ln(HR)] = {results['SE_ln_HR']:.4f}")
    print(f"z_HR = {results['z_HR']:.3f}")
    print()
    print("-" * 70)
    print("COMPLETE p-fr-nb TRIPLET:")
    print("-" * 70)
    print(f"p-value = {results['p']:.6f}  →  {'SIGNIFICANT' if results['p']<=ALPHA else 'NOT SIGNIFICANT'} at α = 0.05")
    print(f"fr: SFQ = {results['SFQ']:.6f}  (fragility: stability of classification)")
    print(f"nb: SRQ = {results['SRQ']:.6f}  (robustness: distance from neutrality)")
    print()

    # Interpretation guidance
    print("-" * 70)
    print("INTERPRETATION:")
    print("-" * 70)

    # Fragility interpretation
    if results['SFQ'] < 0.10:
        fr_interp = "VERY FRAGILE (unstable)"
    elif results['SFQ'] < 0.25:
        fr_interp = "FRAGILE to MILDLY STABLE"
    elif results['SFQ'] < 0.40:
        fr_interp = "MODERATELY STABLE"
    else:
        fr_interp = "VERY STABLE"
    print(f"Fragility: {fr_interp}")

    # Robustness interpretation
    if results['SRQ'] < 0.05:
        nb_interp = "AT NEUTRALITY BOUNDARY"
    elif results['SRQ'] < 0.10:
        nb_interp = "NEAR NEUTRALITY"
    elif results['SRQ'] < 0.25:
        nb_interp = "MODERATE DISTANCE"
    elif results['SRQ'] < 0.50:
        nb_interp = "CLEAR SEPARATION"
    else:
        nb_interp = "FAR FROM NEUTRALITY"
    print(f"Robustness: {nb_interp}")

    # Overall evidence tier (for claims of "treatment benefit")
    # HR < 1 = benefit, HR > 1 = harm
    if results['p'] <= ALPHA:
        if results['SFQ'] >= 0.40 and results['SRQ'] >= 0.50:
            tier = "★★★★★ GOLD-STANDARD (highly convincing)"
        elif results['SFQ'] >= 0.25 and results['SRQ'] >= 0.25:
            tier = "★★★★ STRONG"
        elif results['SFQ'] >= 0.10 and results['SRQ'] >= 0.10:
            tier = "★★★ MODERATE"
        else:
            tier = "★★ WEAK/DISCORDANT (significant but fragile and/or near neutrality)"
    else:
        if results['SFQ'] >= 0.25 and results['SRQ'] <= 0.10:
            tier = "★ CREDIBLE NEGATIVE (stable null, near neutrality)"
        else:
            tier = "NOT SIGNIFICANT (interpret based on context)"

    # Direction note
    if results['HR'] < 1:
        direction = "HR < 1 suggests treatment BENEFIT (reduced hazard)"
    elif results['HR'] > 1:
        direction = "HR > 1 suggests treatment HARM (increased hazard)"
    else:
        direction = "HR = 1 indicates NO EFFECT"

    print(f"\nDirection: {direction}")
    print(f"Evidence tier: {tier}")
    print("="*70 + "\n")

# ==================== Example usage ====================
if __name__ == "__main__":
    print("\n" + "="*70)
    print("Survival Outcomes Calculator – FRAGILITY_METRICS v10.2")
    print("For overall survival, PFS, time-to-event analyses")
    print("="*70 + "\n")

    # Example 1: ABC-AF trial (composite stroke or death)
    print("EXAMPLE 1: ABC-AF trial (primary outcome: stroke or death)")
    print("Reference: Guo et al. Circulation 2025")
    print("ABC-AF strategy vs standard care")
    res1 = calculate_survival_metrics(HR=1.19, ci_lower=0.96, ci_upper=1.48)
    print_results(res1)

    # Example 2: Hypothetical strong survival benefit
    print("\n" + "="*70)
    print("EXAMPLE 2: Hypothetical strong survival benefit")
    res2 = calculate_survival_metrics(HR=0.65, ci_lower=0.52, ci_upper=0.81)
    print_results(res2)

    # Example 3: Fragile result (barely significant)
    print("\n" + "="*70)
    print("EXAMPLE 3: Hypothetical fragile result (barely significant)")
    res3 = calculate_survival_metrics(HR=0.85, ci_lower=0.73, ci_upper=0.99)
    print_results(res3)

    # Example 4: Non-significant with clear separation (underpowered)
    print("\n" + "="*70)
    print("EXAMPLE 4: Non-significant but clear separation (underpowered study)")
    res4 = calculate_survival_metrics(HR=0.70, ci_lower=0.45, ci_upper=1.09)
    print_results(res4)

    # Interactive mode
    print("\n" + "="*70)
    print("INTERACTIVE MODE")
    print("="*70)

    try:
        use_interactive = input("\nRun interactive mode? (y/n): ").lower()

        if use_interactive == 'y':
            print("\nEnter survival outcome data:")
            print("(Typically reported as: 'HR = X.XX, 95% CI X.XX-X.XX, p = X.XX')")

            HR = float(input("\nHazard ratio (HR): "))
            ci_lower = float(input("95% CI lower bound: "))
            ci_upper = float(input("95% CI upper bound: "))

            results = calculate_survival_metrics(HR, ci_lower, ci_upper)
            print_results(results)

    except KeyboardInterrupt:
        print("\n\nExiting...")
    except Exception as e:
        print(f"\nError: {e}")
        print("Please check your inputs and try again.")

print("\n" + "="*70)
print("Citation:")
print("Heston, T. F. (2025). Fragility Metrics Toolkit [Software].")
print("Zenodo. https://doi.org/10.5281/zenodo.17254763")
print("="*70 + "\n")